source: https://github.com/minoguep/rhyme_detection and https://paulminogue.com/index.php/2021/02/14/using-a-siamese-neural-network-to-create-a-simple-rhyme-detector/

In [2]:
import string
import json

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Subtract
from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, CSVLogger, EarlyStopping

tqdm.pandas()

MAX_LEN = 64
SEED = 420
# sets random, np.random and tf.random seed
tf.keras.utils.set_random_seed(
    SEED
)

2022-04-03 00:03:28.903884: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-03 00:03:28.903924: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Step 1: Create dataset
We want equally many positive and negative samples of rhyme pairs

In [3]:
rhyme_pairs = pd.read_csv("rhyme_word_pairs.tsv", sep="\t",)
pos = rhyme_pairs.loc[rhyme_pairs["rhyme"]==1].copy()
neg = rhyme_pairs.loc[rhyme_pairs["rhyme"]==0].copy()
neg = neg.sample(n=len(pos), random_state=SEED)
df = pd.concat([pos, neg])
df

,word_a,word_b,rhyme
1,fantasier,melodier,1
2,fullkommenhet,atlet,1
3,heder,der,1
6,hugger,vugger,1
20,pergamenter,argumenter,1
...,...,...,...
8403,griper,lenger,0
8178,ett,kast,0
2303,ord,art,0
27218,ikke,trette,0


In [4]:
all_data = pd.read_csv("tita_rhymes_poems.tsv", sep="\t")

all_text = ""
for e in all_data.stanza:
    all_text += e

### Step 2: Create model
Copy paste from Pauls notebook

In [5]:
def tokenize_inputs(phrase_a, phrase_b, tokenizer):
    tokenized_phrases = tokenizer.texts_to_sequences([phrase_a, phrase_b])

    # now loop through inputs and pad or reduce size if required
    tokenized_phrases_for_output = []
    for phrase in tokenized_phrases:
        if len(phrase) < MAX_LEN:
            length_to_pad = MAX_LEN - len(phrase)
            phrase_for_output = ([0] * length_to_pad) + phrase
        elif len(phrase) > MAX_LEN:
            phrase_for_output = phrase[-MAX_LEN:]
        else:
            phrase_for_output = phrase
        tokenized_phrases_for_output.append(phrase_for_output)

    return tf.constant(tokenized_phrases_for_output, dtype=tf.float64)

In [6]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(all_text)

df['word_tokens'] = df.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

tokenizer_config = tokenizer.to_json()

with open('tokenizer_config.json', 'w') as f:
    f.write(tokenizer_config)

  0%|          | 0/14512 [00:00<?, ?it/s]

2022-04-03 00:03:31.671781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-03 00:03:31.671809: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-03 00:03:31.671829: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tita-laptop): /proc/driver/nvidia/version does not exist
2022-04-03 00:03:31.672794: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
def create_model():
  
    word_a_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_a_input_tokens'
      )
    word_b_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_b_input_tokens'
      )
    
    # This is the siamese portion of the model 
    common_lstm = LSTM(64, return_sequences=False, activation="relu", name="common_lstm_layer")

    word_a_lstm_output = common_lstm(word_a_input_tokens)
    word_b_lstm_output = common_lstm(word_b_input_tokens)

    #concatenate_lstm_outputs
    concat_layer = Subtract(name="concatenate_lstm_outputs")(
      [word_a_lstm_output, word_b_lstm_output]
      )
    
    # dense layers before final classification
    dense_layers = Dense(64, activation="relu", name="first_dense_layer")(concat_layer)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(32, activation="relu", name="second_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(8, activation="relu", name="third_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    classification_layer = Dense(1, activation="sigmoid", name="classification_layer")(dense_layers)
    
    model = Model(
      inputs=[word_a_input_tokens, word_b_input_tokens], 
      outputs = classification_layer
      )

    model.compile(
      loss="binary_crossentropy",
      metrics=["accuracy"],
      optimizer="Adam"
    )

    return model

In [8]:
X_train_indexes, X_test_indexes, y_train, y_test = train_test_split(
    list(df.index), list(df['rhyme']), stratify=df['rhyme'], 
    test_size=0.4, random_state=SEED
    )

X_test_indexes, X_val_indexes, y_test, y_val = train_test_split(
    X_test_indexes, y_test, stratify=y_test, 
    test_size=0.25, random_state=SEED
    )

X_train = tf.convert_to_tensor(list(df.loc[X_train_indexes]["word_tokens"]))
X_val = tf.convert_to_tensor(list(df.loc[X_val_indexes]["word_tokens"]))
X_test = tf.convert_to_tensor(list(df.loc[X_test_indexes]["word_tokens"]))

y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

In [9]:
print(f"""
    Data set size: Full set: {len(df)}
    Train: {len(X_train)}
    Validation: {len(X_val)}
    Test: {len(X_test)}
""")



    Data set size: Full set: 14512
    Train: 8707
    Validation: 1452
    Test: 4353



In [10]:
model_name = "rhyme_model1"

In [11]:
# #Uncomment to train model 

# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train[:, 0], X_train[:, 1]],
#     y_train,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

In [12]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94      2176
           1       0.92      0.96      0.94      2177

    accuracy                           0.94      4353
   macro avg       0.94      0.94      0.94      4353
weighted avg       0.94      0.94      0.94      4353



In [13]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)

print(sample_tokens[:, 0].shape, sample_tokens[:, 1].shape)

sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Sentence 1: {samples[i][0]}")
    print(f"Sentence 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

(7, 64) (7, 64)
Sentence 1: Kan du ikke se det
Sentence 2: Deg skal jeg lede
Rhyme(0.6941999793052673)
---------------

Sentence 1: Kaker av alle slag
Sentence 2: Her henger Norges flagg
Rhyme(0.8702999949455261)
---------------

Sentence 1: Jeg har ikke tid
Sentence 2: Til dette svineri
Rhyme(0.7032999992370605)
---------------

Sentence 1: Hva har du sagt
Sentence 2: Kaken er bakt
Rhyme(0.9254999756813049)
---------------

Sentence 1: Barna er lagt
Sentence 2: Kaken er laget
Non-rhyme(0.000699999975040555)
---------------

Sentence 1: Gjorde du det med vilje
Sentence 2: Kaken smaker vanilje
Rhyme(0.9366000294685364)
---------------

Sentence 1: Dette vokser
Sentence 2: Satans underbukser
Rhyme(0.9340999722480774)
---------------



## Try again with mirrored examples included

In [14]:
def get_mirrored_df(df):
    mirror = pd.DataFrame({"word_a": df["word_b"], 
                           "word_b": df["word_a"], 
                           "rhyme": df["rhyme"], 
                           "word_tokens":[(t[1], t[0]) for t in df["word_tokens"]]})
    return pd.concat((df, mirror))

In [15]:
train_df = df.loc[X_train_indexes]
dev_df = df.loc[X_val_indexes]
len(train_df), len(dev_df)

(8707, 1452)

In [16]:
double_train = get_mirrored_df(train_df)
double_train

,word_a,word_b,rhyme,word_tokens
8032,venn,formår,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
24484,ville,Røst,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
4255,elven,støv,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
24330,båsen,tomme,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
7665,slaver,tror,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
...,...,...,...,...
14350,støv,ung,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
19712,dødningdrakt,prakt,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
19219,årevis,flis,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
22260,bue,strand,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."


In [17]:
double_dev = get_mirrored_df(dev_df)
double_dev

,word_a,word_b,rhyme,word_tokens
25195,leve,vann,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
8166,jord,ror,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
12891,før,snakk,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
23019,fargespill,mild,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
26071,grisen,ekspeditrisen,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
...,...,...,...,...
17849,før,tørr,1,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
11377,hånd,ild,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
16025,fortell,herskerinnen,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."
28151,skatt,befale,0,"((tf.Tensor(0.0, shape=(), dtype=float64), tf...."


In [18]:
X_train = tf.convert_to_tensor(list(double_train["word_tokens"]))
X_val = tf.convert_to_tensor(list(double_dev["word_tokens"]))

y_train = tf.convert_to_tensor(list(double_train["rhyme"]))
y_val = tf.convert_to_tensor(list(double_dev["rhyme"]))

In [19]:
model_name = "rhyme_model2"

In [20]:
#Uncomment to train model 
# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train[:, 0], X_train[:, 1]],
#     y_train,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

In [21]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      2176
           1       0.97      0.93      0.95      2177

    accuracy                           0.95      4353
   macro avg       0.95      0.95      0.95      4353
weighted avg       0.95      0.95      0.95      4353



In [22]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)

print(sample_tokens[:, 0].shape, sample_tokens[:, 1].shape)

sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Sentence 1: {samples[i][0]}")
    print(f"Sentence 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

(7, 64) (7, 64)
Sentence 1: Kan du ikke se det
Sentence 2: Deg skal jeg lede
Non-rhyme(0.14920000731945038)
---------------

Sentence 1: Kaker av alle slag
Sentence 2: Her henger Norges flagg
Rhyme(0.9937000274658203)
---------------

Sentence 1: Jeg har ikke tid
Sentence 2: Til dette svineri
Non-rhyme(0.25589999556541443)
---------------

Sentence 1: Hva har du sagt
Sentence 2: Kaken er bakt
Rhyme(0.9451000094413757)
---------------

Sentence 1: Barna er lagt
Sentence 2: Kaken er laget
Non-rhyme(0.01209999993443489)
---------------

Sentence 1: Gjorde du det med vilje
Sentence 2: Kaken smaker vanilje
Rhyme(0.9998000264167786)
---------------

Sentence 1: Dette vokser
Sentence 2: Satans underbukser
Rhyme(0.9998000264167786)
---------------



# Try again with different 2:3 positive to negative ratio
Use same test and dev sets, and same tokenizer. Only expand training set.  
Train set is already 50/50 positive and negative. We want to make it 40/60 --> increase negative examples by half of what we already have

In [23]:
new_neg = len(neg) // 2
new_neg

3628

In [24]:
neg = rhyme_pairs.loc[rhyme_pairs["rhyme"]==0].copy()
# use same seed, and extract all the pairs already used + the ones we need 
negative2 = neg.sample(n=len(pos) + new_neg, random_state=SEED)

In [25]:
unused_neg = negative2[len(pos):]
unused_neg

,word_a,word_b,rhyme
13954,fri,mere,0
3616,tinder,bedøvet,0
8417,ny,vann,0
16469,gret,visste,0
15765,Ola,laks,0
...,...,...,...
2735,geseller,skam,0
19487,like,merke,0
249,sinn,ham,0
2445,sinn,kvist,0


In [26]:
# use same tokenizer as above
unused_neg['word_tokens'] = unused_neg.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

unused_neg = get_mirrored_df(unused_neg)

X_train_new = tf.convert_to_tensor(list(X_train) + list(unused_neg["word_tokens"]))
y_train_new = tf.convert_to_tensor(list(y_train) + list(unused_neg["rhyme"]))


  0%|          | 0/3628 [00:00<?, ?it/s]

/tmp/ipykernel_13850/2379011568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unused_neg['word_tokens'] = unused_neg.progress_apply(


In [27]:
X_train_new.shape

TensorShape([24670, 2, 64])

In [28]:
model_name = "rhyme_model_23"

In [29]:
# # Uncomment to train model
# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train_new[:, 0], X_train_new[:, 1]],
#     y_train_new,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

In [30]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2176
           1       0.97      0.95      0.96      2177

    accuracy                           0.96      4353
   macro avg       0.96      0.96      0.96      4353
weighted avg       0.96      0.96      0.96      4353



In [55]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)
sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Lyric 1: {samples[i][0]}")
    print(f"Lyric 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

Lyric 1: Kan du ikke se det
Lyric 2: Deg skal jeg lede
Rhyme(0.9348999857902527)
---------------

Lyric 1: Kaker av alle slag
Lyric 2: Her henger Norges flagg
Non-rhyme(0.2143000066280365)
---------------

Lyric 1: Jeg har ikke tid
Lyric 2: Til dette svineri
Rhyme(0.9348999857902527)
---------------

Lyric 1: Hva har du sagt
Lyric 2: Kaken er bakt
Rhyme(0.9348999857902527)
---------------

Lyric 1: Barna er lagt
Lyric 2: Kaken er laget
Non-rhyme(0.0)
---------------

Lyric 1: Gjorde du det med vilje
Lyric 2: Kaken smaker vanilje
Rhyme(0.9348999857902527)
---------------

Lyric 1: Dette vokser
Lyric 2: Satans underbukser
Rhyme(0.849399983882904)
---------------



## Use all rhyme pairs

In [41]:
unused_neg_2 = neg.sample(n=len(neg), random_state=SEED)[len(negative2):]

In [42]:
# use same tokenizer as above
unused_neg_2['word_tokens'] = unused_neg_2.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

unused_neg_2 = get_mirrored_df(unused_neg_2)

X_train_new = tf.convert_to_tensor(list(X_train_new) + list(unused_neg_2["word_tokens"]))
y_train_new = tf.convert_to_tensor(list(y_train_new) + list(unused_neg_2["rhyme"]))
X_train_new.shape

  0%|          | 0/11563 [00:00<?, ?it/s]

TensorShape([47796, 2, 64])

In [43]:
model_name = "rhyme_model_all_pairs"

In [44]:
# # Uncomment to train model
# model = create_model()

# model_checkpoint = ModelCheckpoint(f"models/{model_name}.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger(f'logs/training_{model_name}.log')
# early_stop = EarlyStopping(monitor='loss', patience=5)

# history = model.fit(
#     [X_train_new[:, 0], X_train_new[:, 1]],
#     y_train_new,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger, early_stop],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

Epoch 1/100
374/374 [==============================] - 17s 40ms/step - loss: 0.3816 - accuracy: 0.8105 - val_loss: 0.5525 - val_accuracy: 0.5000
Epoch 2/100
374/374 [==============================] - 14s 39ms/step - loss: 0.2974 - accuracy: 0.8657 - val_loss: 0.5257 - val_accuracy: 0.8592
Epoch 3/100
374/374 [==============================] - 16s 42ms/step - loss: 0.2740 - accuracy: 0.9289 - val_loss: 0.4851 - val_accuracy: 0.8674
Epoch 4/100
374/374 [==============================] - 17s 44ms/step - loss: 0.2543 - accuracy: 0.9320 - val_loss: 0.4277 - val_accuracy: 0.8922
Epoch 5/100
374/374 [==============================] - 16s 43ms/step - loss: 0.2384 - accuracy: 0.9321 - val_loss: 0.3989 - val_accuracy: 0.8988
Epoch 6/100
374/374 [==============================] - 15s 41ms/step - loss: 0.2307 - accuracy: 0.9332 - val_loss: 0.3711 - val_accuracy: 0.9170
Epoch 7/100
374/374 [==============================] - 17s 46ms/step - loss: 0.2228 - accuracy: 0.9329 - val_loss: 0.3498 - val_ac

In [45]:
# load the model
model = load_model(f"models/{model_name}.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      2176
           1       0.99      0.93      0.96      2177

    accuracy                           0.96      4353
   macro avg       0.96      0.96      0.96      4353
weighted avg       0.96      0.96      0.96      4353



## Try to use whole sentences instead of just word pairs

In [ ]:
# df.to_csv("sentence_pair_rhyme.tsv", sep="\t", index=False)

In [ ]:
# samples = [
#     ["Kan du ikke se det?", "Deg skal jeg lede"], 
#     ["Kaker av alle slag", "Her henger Norges flagg"], 
#     ["Jeg har ikke tid", "Til dette svineri"],
#     ["Hva har du sagt", "Kaken er bakt"], 
#     ["Barna er lagt", "Kaken er laget"],
#     ["Er du sikker på at dette er med vilje?", "Hjertet ditt smaker vanilje"], 
#     ["Dette vokser", "Satans underbukser"],
# ]

# sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
# sample_tokens = tf.convert_to_tensor(sample_tokens)
# sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
# predictions = [round(pred[0], 4) for pred in sample_pred]
# for i in range(len(samples)):
#     print(f"Lyric 1: {samples[i][0]}")
#     print(f"Lyric 2: {samples[i][1]}")
#     print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
#     print("---------------\n")